# Kuhifadhi Hoteli na Middleware ya Mwanachama wa Kipaumbele

Notebook hii inaonyesha **middleware inayotegemea kazi** kwa kutumia Mfumo wa Microsoft Agent. Tunajenga juu ya mfano wa mtiririko wa kazi wa masharti kwa kuongeza safu ya middleware inayowapa wanachama wa kipaumbele haki maalum.

## Kile Utakachojifunza:
1. **Middleware Inayotegemea Kazi**: Kukamata na kurekebisha matokeo ya kazi
2. **Ufikiaji wa Muktadha**: Kusoma na kurekebisha `context.result` baada ya utekelezaji
3. **Utekelezaji wa Mantiki ya Biashara**: Faida za wanachama wa kipaumbele
4. **Kubadilisha Matokeo**: Kubadilisha matokeo ya kazi kulingana na hadhi ya mtumiaji
5. **Mtiririko Ule Ule, Matokeo Tofauti**: Mabadiliko yanayoendeshwa na middleware

## Muundo wa Mtiririko wa Kazi na Middleware:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## Tofauti Muhimu kutoka kwa Mtiririko wa Kazi wa Masharti:

**Bila Middleware** (14-conditional-workflow.ipynb):
- Paris haina vyumba → Elekeza kwa alternative_agent

**Na Middleware** (notebook hii):
- Mtumiaji wa kawaida + Paris → Hakuna vyumba → Elekeza kwa alternative_agent
- Mtumiaji wa kipaumbele + Paris → 🌟 Middleware inabadilisha! → Vyumba vinapatikana → Elekeza kwa booking_agent

## Mahitaji:
- Mfumo wa Microsoft Agent umesakinishwa
- Uelewa wa mtiririko wa kazi wa masharti (angalia 14-conditional-workflow.ipynb)
- Token ya GitHub au funguo ya API ya OpenAI
- Uelewa wa msingi wa mifumo ya middleware


In [2]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Hatua ya 1: Fafanua Miundo ya Pydantic kwa Matokeo Yaliyopangiliwa

Miundo hii inaelezea **schema** ambayo mawakala watarudisha. Tumeongeza sehemu ya `priority_override` ili kufuatilia wakati middleware inabadilisha matokeo ya upatikanaji.


In [3]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Hatua ya 2: Fafanua Hifadhidata ya Wanachama wa Kipaumbele

Kwa mfano huu, tutatengeneza hifadhidata ya wanachama wa kipaumbele. Katika uzalishaji, hii ingeuliza hifadhidata halisi au API.

**Wanachama wa Kipaumbele:**
- `alice@example.com` - Mwanachama wa VIP
- `bob@example.com` - Mwanachama wa Premium  
- `priority_user` - Akaunti ya majaribio


In [4]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## Hatua ya 3: Tengeneza Zana ya Kuhifadhi Hoteli

Sawa na mtiririko wa kazi wa masharti, lakini sasa itazuiliwa na middleware!


In [5]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Hatua ya 4: 🌟 Unda Middleware ya Ukaguzi wa Kipaumbele (SIFA MUHIMU!)

Hii ndiyo **utendaji wa msingi** wa daftari hili. Middleware:

1. **Inazuia** mwito wa kazi ya hotel_booking
2. **Inatekeleza** kazi kawaida kwa kuita `next(context)`
3. **Inachunguza** matokeo katika `context.result`
4. **Inabadilisha** matokeo ikiwa mtumiaji ana kipaumbele na hakuna vyumba vilivyopo
5. **Inarudisha** matokeo yaliyorekebishwa kwa wakala

**Mfumo Muhimu:**
```python
async def my_middleware(context, next):
    await next(context)  # Execute function
    # Now context.result contains the function's output
    if some_condition:
        context.result = new_value  # Override!
```


In [6]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## Hatua ya 5: Fafanua Kazi za Masharti kwa Usambazaji

Kazi za masharti ni sawa na zile za mtiririko wa masharti - zinachunguza matokeo yaliyopangwa ili kuamua usambazaji.


In [7]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## Hatua ya 6: Unda Mtekelezaji wa Maonyesho Maalum

Mtekelezaji sawa na wa awali - unaonyesha matokeo ya mwisho ya mtiririko wa kazi.


In [8]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## Hatua ya 7: Pakia Vigezo vya Mazingira

Sanidi mteja wa LLM (GitHub Models au OpenAI).


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## Hatua ya 8: Unda Mawakala wa AI kwa Middleware

**TOFAUTI MUHIMU:** Wakati wa kuunda availability_agent, tunapitia kipengele cha `middleware`!

Hivi ndivyo tunavyoweka priority_check_middleware kwenye mchakato wa mwito wa kazi wa wakala.


In [ ]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Hatua ya 9: Jenga Mtiririko wa Kazi

Muundo wa mtiririko wa kazi ni sawa na ule wa awali - uelekezaji wa masharti kulingana na upatikanaji.


In [12]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## Hatua ya 10: Jaribio la Kesi 1 - Mtumiaji wa Kawaida huko Paris (Hakuna Ubadilishaji)

Mtumiaji wa kawaida anajaribu kuhifadhi Paris → Hakuna vyumba → Inapelekwa kwa alternative_agent


In [13]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## Hatua ya 11: Jaribio la Kesi 2 - 🌟 Mtumiaji wa Kipaumbele Paris (NA Ubadilishaji!)

Mwanachama wa kipaumbele anajaribu kuhifadhi Paris → Hakuna vyumba mwanzoni → 🌟 Middleware inabadilisha! → Inapeleka kwa booking_agent

**Hii ni onyesho muhimu la nguvu ya middleware!**


In [14]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## Hatua ya 12: Jaribio la Kesi 3 - Mtumiaji wa Kipaumbele Stockholm (Tayari Inapatikana)

Mtumiaji wa kipaumbele anajaribu Stockholm → Vyumba vinapatikana → Hakuna hitaji la kuingilia kati → Inapelekwa kwa booking_agent

Hii inaonyesha kwamba middleware hufanya kazi tu inapohitajika!


In [15]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## Mambo Muhimu na Dhana za Middleware

### ✅ Ulichojifunza:

#### **1. Muundo wa Middleware Inayotegemea Kazi**

Middleware inachukua hatua katikati ya miito ya kazi kwa kutumia kazi rahisi ya async:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Before function execution
    print("Intercepting...")
    
    # Execute the function
    await next(context)
    
    # After function execution - inspect result
    if context.result:
        # Modify result if needed
        context.result = modified_value
```

#### **2. Ufikiaji wa Muktadha na Kubadilisha Matokeo**

- `context.function` - Fikia kazi inayoitwa
- `context.arguments` - Soma hoja za kazi
- `context.kwargs` - Fikia vigezo vya ziada
- `await next(context)` - Tekeleza kazi
- `context.result` - Soma/badilisha matokeo ya kazi

#### **3. Utekelezaji wa Mantiki ya Biashara**

Middleware yetu inatekeleza faida za wanachama wa kipaumbele:
- **Watumiaji wa kawaida**: Hakuna mabadiliko, mtiririko wa kawaida
- **Watumiaji wa kipaumbele**: Badilisha "hakuna upatikanaji" → "inapatikana"
- **Mantiki ya masharti**: Hubadilisha tu inapohitajika

#### **4. Mtiririko Ule Ule, Matokeo Tofauti**

Nguvu ya middleware:
- ✅ Hakuna mabadiliko kwenye muundo wa mtiririko wa kazi
- ✅ Hakuna mabadiliko kwenye kazi ya zana
- ✅ Hakuna mabadiliko kwenye mantiki ya uelekezaji wa masharti
- ✅ Middleware tu → Tabia tofauti!

### 🚀 Matumizi ya Kwenye Ulimwengu Halisi:

1. **Vipengele vya VIP/Premium**
   - Badilisha mipaka ya kiwango kwa watumiaji wa premium
   - Toa upatikanaji wa kipaumbele kwa rasilimali
   - Fungua vipengele vya premium kwa njia ya nguvu

2. **Upimaji wa A/B**
   - Elekeza watumiaji kwenye utekelezaji tofauti
   - Jaribu vipengele vipya na watumiaji maalum
   - Utoaji wa vipengele kwa hatua

3. **Usalama na Uzingatiaji**
   - Kagua miito ya kazi
   - Zuia operesheni nyeti
   - Tekeleza sheria za biashara

4. **Uboreshaji wa Utendaji**
   - Hifadhi matokeo kwa watumiaji maalum
   - Ruka operesheni ghali inapowezekana
   - Ugawaji wa rasilimali kwa njia ya nguvu

5. **Ushughulikiaji wa Makosa na Jaribio Tena**
   - Kamata na ushughulikie makosa kwa ustadi
   - Tekeleza mantiki ya jaribio tena
   - Rudi kwenye utekelezaji mbadala

6. **Kumbukumbu na Ufuatiliaji**
   - Fuatilia muda wa utekelezaji wa kazi
   - Rekodi vigezo na matokeo
   - Fuatilia mifumo ya matumizi

### 🔑 Tofauti Muhimu na Mapambo:

| Kipengele | Mapambo | Middleware |
|-----------|---------|------------|
| **Wigo** | Kazi moja | Kazi zote katika wakala |
| **Urahisi** | Imekaa wakati wa ufafanuzi | Inabadilika wakati wa utekelezaji |
| **Muktadha** | Uliopunguzwa | Muktadha kamili wa wakala |
| **Muundo** | Mapambo mengi | Njia ya middleware |
| **Ufahamu wa Wakala** | Hapana | Ndio (ufikiaji wa hali ya wakala) |

### 📚 Wakati wa Kutumia Middleware:

✅ **Tumia middleware wakati:**
- Unahitaji kubadilisha tabia kulingana na hali ya mtumiaji/kikao
- Unataka kutumia mantiki kwa kazi nyingi
- Unahitaji ufikiaji wa muktadha wa kiwango cha wakala
- Unatekeleza masuala ya msalaba (kumbukumbu, uthibitisho, nk.)

❌ **Usitumie middleware wakati:**
- Uthibitishaji rahisi wa pembejeo (tumia Pydantic)
- Mantiki maalum ya kazi (weka ndani ya kazi)
- Mabadiliko ya mara moja (badilisha tu kazi)

### 🎓 Miundo ya Juu:

```python
# Multiple middleware (execution order matters!)
middleware=[
    logging_middleware,      # Logs first
    auth_middleware,         # Then checks auth
    cache_middleware,        # Then checks cache
    rate_limit_middleware,   # Then rate limits
    priority_check_middleware  # Finally priority check
]

# Conditional middleware execution
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Modify result
    else:
        # Skip execution entirely
        context.result = cached_value
```

### 🔗 Dhana Zinazohusiana:

- **Middleware ya Wakala**: Inachukua hatua katikati ya miito ya agent.run()
- **Middleware ya Kazi**: Inachukua hatua katikati ya miito ya kazi ya zana (tuliyotumia!)
- **Njia ya Middleware**: Mlolongo wa middleware unaotekelezwa kwa mpangilio
- **Uenezaji wa Muktadha**: Pitisha hali kupitia mlolongo wa middleware



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kwa usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
